In [1]:
import math
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import skimage
import pickle as pkl
import cv2
import tensorflow.python.platform
from keras.preprocessing import sequence
from collections import Counter

C:\Users\b160544me\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\b160544me\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\b160544me\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\b160544me\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: Futu

In [10]:
#Give csv of the dataset for which feature extraction has to be performed
df = pd.read_csv('C:/Users/b160544me/Desktop/siva/project/dataset/VQAMed2019Validation/QAPairsByCategory/C4_Abnormality_val.txt',sep='|',header=None)

In [11]:
df=df[[0,1]]
df.head(5)

,0,1
0,synpic54733,what is the primary abnormality in this image?
1,synpic25647,is there an abnormality in the x-ray?
2,synpic35681,is there an abnormality in the mri?
3,synpic39641,what is the primary abnormality in this image?
4,synpic35693,is this a normal mri?


In [12]:
df.columns=['imageno','question']
print(len(df))
df.head(25)


500


,imageno,question
0,synpic54733,what is the primary abnormality in this image?
1,synpic25647,is there an abnormality in the x-ray?
2,synpic35681,is there an abnormality in the mri?
3,synpic39641,what is the primary abnormality in this image?
4,synpic35693,is this a normal mri?
5,synpic17419,is this a normal ct scan?
6,synpic32909,what abnormality is seen in the image?
7,synpic48040,is there an abnormality in the ct scan?
8,synpic48351,what abnormality is seen in the image?
9,synpic49503,is there evidence of any abnormalities?


In [13]:
# to remove inputs where answers span across multiple columns  
#boolmat=[]
#for i in range(0,len(df)):
#    boolmat.append(df['question'].iloc[i][len(df['question'].iloc[i])-1]=='?')
#df=df[boolmat]

In [14]:
#path to VGG model 
vgg_path = 'C:/Users/b160544me/Desktop/siva/project/code/vgg16-20160129.tfmodel'
#path to directory that contains images
img_path='C:/Users/b160544me/Desktop/siva/project/dataset/VQAMed2019Validation/Val_images'
def crop_image(x, target_height=227, target_width=227, as_float=True):
    image = cv2.imread(x)
    if as_float:
        image = image.astype(np.float32)

    if len(image.shape) == 2:
        image = np.tile(image[:,:,None], 3)
    elif len(image.shape) == 4:
        image = image[:,:,:,0]

    height, width, rgb = image.shape
    if width == height:
        resized_image = cv2.resize(image, (target_height,target_width))

    elif height < width:
        resized_image = cv2.resize(image, (int(width * float(target_height)/height), target_width))
        cropping_length = int((resized_image.shape[1] - target_height) / 2)
        resized_image = resized_image[:,cropping_length:resized_image.shape[1] - cropping_length]

    else:
        resized_image = cv2.resize(image, (target_height, int(height * float(target_width) / width)))
        cropping_length = int((resized_image.shape[0] - target_width) / 2)
        resized_image = resized_image[cropping_length:resized_image.shape[0] - cropping_length,:]

    return cv2.resize(resized_image, (target_height, target_width))

def read_image(path):
    img = crop_image(path, target_height=224, target_width=224)
    if img.shape[2] == 4:
        img = img[:,:,:3]
    img = img[None, ...]
    return img

tf.reset_default_graph()
with open(vgg_path,'rb') as f:
        fileContent = f.read()
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(fileContent)
final_array_test = []
images = tf.placeholder("float32", [1, 224, 224, 3])
tf.import_graph_def(graph_def, input_map={"images":images})
graph = tf.get_default_graph()
    
with tf.Session() as sess:    
    for i in df.imageno:
        feat = read_image(img_path+"/"+i+".jpg")
        #Here activations extracted from the last fully connected layers are used as features
        fc7 = sess.run(graph.get_tensor_by_name("import/Relu_1:0"), feed_dict={images:feat})      
        fc7 = np.squeeze(fc7)
        final_array_test.append(fc7)



In [15]:
len(final_array_test)

500

In [16]:
#Saving the features extracted for the images
import pickle
pickle.dump(final_array_test, open( "C:/Users/b160544me/Desktop/siva/project/image_feature_test.pkl", "wb" ) )